<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/af_relax_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#af_relax_design (WIP)
Backprop through AlphaFold for backbone hallucination, followed by redesign wih ProteinMPNN

This notebook implements the protocol described in:

**Efficient and scalable de novo protein design using a relaxed sequence space**

Christopher Josef Frank, Ali Khoshouei, Yosta de Stigter, Dominik Schiewitz, Shihao Feng, Sergey Ovchinnikov, Hendrik Dietz

doi: https://doi.org/10.1101/2023.02.24.529906

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.0")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

## hallucination
For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).

In [ ]:
import jax
import jax.numpy as jnp
from colabdesign.af.alphafold.common import residue_constants

def rg_loss(inputs, outputs):
  positions = outputs["structure_module"]["final_atom_positions"]
  ca = positions[:,residue_constants.atom_order["CA"]]
  center = ca.mean(0)
  rg = jnp.sqrt(jnp.square(ca - center).sum(-1).mean() + 1e-8)
  rg_th = 2.38 * ca.shape[0] ** 0.365
  rg = jax.nn.elu(rg - rg_th)
  return {"rg":rg}

In [ ]:
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", loss_callback=rg_loss)
af_model.opt["weights"]["rg"] = 0.1
af_model.prep_inputs(length=100, copies=1)

print("length",af_model._len)

In [ ]:
# initialize sequence gumbel+softmax
af_model.restart(mode=["gumbel","soft"])
af_model.set_weights(plddt=1.0, pae=1.0, rg=0.1, con=1.0)
af_model.design_logits(50)

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()

#Redesign with ProteinMPNN

In [ ]:
from colabdesign.mpnn import mk_mpnn_model
if "mpnn_model" not in dir():
  mpnn_model = mk_mpnn_model()

mpnn_model.prep_inputs(pdb_filename=f"{af_model.protocol}.pdb")
out = mpnn_model.sample(num=1, batch=8)

In [ ]:
out["seq"]